In [ ]:
from fastai2.torch_basics import *
from fastai2.data.all import *
from fastai2.vision.core import *

In [ ]:
source = untar_data(URLs.MNIST_TINY)/'train'
items = get_image_files(source)
fn = items[0]
fn

Path('/home/quantran/.fastai/data/mnist_tiny/train/7/9286.png')

In [ ]:
type(array(3))

numpy.ndarray

In [ ]:
def neg(x): return -x

In [ ]:
neg((1,2))

TypeError: bad operand type for unary -: 'tuple'

In [ ]:
neg(array((1,2)))

array([-1, -2])

# Transform decorator can make a function work in a flexible way (inputs with different type)

Class decorator

In [ ]:
@Transform
def neg(x): return -x

This @Transform looks like you gonna pass the func (neg) in ```__init__``` of class Transform

In [ ]:
# equivalent
def _neg(x): return -x
neg = Transform(_neg)

In [ ]:
neg((1,2)) # this is like calling encode function in Transform class

(-1, -2)

In [ ]:
img = load_image(fn)
img

In [ ]:
img.shape

(28, 28)

In [ ]:
type(img)

PIL.Image.Image

In [ ]:
t = tensor(array(img))
t[12:14]

tensor([[  0,   0,   0,   0,   0,   0,   0,   0, 164, 239,  26,   0,   0,   0,
           0,  96, 236,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,  12,  47,   0,   0,   0,   0,
           0, 179, 235,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]],
       dtype=torch.uint8)

In [ ]:
neg(t[12:14]) # negative here doesn't really work, due to type uint8 doesn't allow negative number

tensor([[  0,   0,   0,   0,   0,   0,   0,   0,  92,  17, 230,   0,   0,   0,
           0, 160,  20,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0, 244, 209,   0,   0,   0,   0,
           0,  77,  21,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]],
       dtype=torch.uint8)

In [ ]:
t_float = t.float()
neg(t_float[12:14])

tensor([[  -0.,   -0.,   -0.,   -0.,   -0.,   -0.,   -0.,   -0., -164., -239.,
          -26.,   -0.,   -0.,   -0.,   -0.,  -96., -236.,   -0.,   -0.,   -0.,
           -0.,   -0.,   -0.,   -0.,   -0.,   -0.,   -0.,   -0.],
        [  -0.,   -0.,   -0.,   -0.,   -0.,   -0.,   -0.,   -0.,  -12.,  -47.,
           -0.,   -0.,   -0.,   -0.,   -0., -179., -235.,   -0.,   -0.,   -0.,
           -0.,   -0.,   -0.,   -0.,   -0.,   -0.,   -0.,   -0.]])

Note that @Transform will reserve type (see below, search for 'reserve')

# Quick note on the use of partial function and decorator

## Partial: to set up certain keyword arguments before hand

In [ ]:
def sum_func(a,b): return a+b

sum_func(1,2)

3

In [ ]:
sum_func2 = partial(sum_func,1) #add kwargs {a:1} to sum_func
sum_func2(2) # only need to provide argument b

3

## Decorator: wrappers to existing functions, to reuse/ add new functionality to existing functions

In [ ]:
def sum_func(a,b): return a+b # the existing function

In [ ]:
def deco(f):
    def _func(a,b): return f(a,b) # does nothing
    return _func

In [ ]:
my_func = deco(sum_func)
my_func(1,2)

3

In [ ]:
def deco(f):
    def _func(a,b): return 1+ f(a,b) # does new thing to existing f, e.g. increase value by 1
    return _func

my_func = deco(sum_func)
my_func(1,2)

4

or use decorator

In [ ]:
@deco
def sum_func(a,b): return a+b

In [ ]:
my_func = sum_func # deco function (with sum_func as parameter) is called, and _func is returned (without any input params)
my_func(1,2) # put params a and b

4

## Combine decorator and partial

To reuse/add new stuff to existing functions and set up certain kwargs before hand

In [ ]:
def deco(f):
    def _func(a,b): return partial(f,a,b) # note that this function returns a partial function, 
    # not a value from a function like above
    return _func

@deco
def sum_func(a,b,c): return a+b+c

In [ ]:
my_func = sum_func(1,2) # deco is called, _func is returned. _func is provided 2 input for parameters a and b
my_func(3) # provide argument c

6

In [ ]:
my_func2 = sum_func(3,3) # my_func2 is a NEW variety of the old function sum_func (because of new kwargs)
my_func2(3)

9

# Patch: add new function (or property aka variables) to existing class using decorator

In [ ]:
def copy_func(f):
    "Copy a non-builtin function (NB `copy.copy` does not work for this)"
    if not isinstance(f,FunctionType): return copy(f)
    fn = FunctionType(f.__code__, f.__globals__, f.__name__, f.__defaults__, f.__closure__)
    fn.__dict__.update(f.__dict__)
    return fn

def patch_to(cls, as_prop=False):
    "Decorator: add `f` to `cls`"
    if not isinstance(cls, (tuple,list)): cls=(cls,)
    def _inner(f):
        # to check when adding new func to class, all the metadata are still correct
        for c_ in cls:
            nf = copy_func(f)
            # `functools.update_wrapper` when passing patched function to `Pipeline`, so we do it manually
            for o in functools.WRAPPER_ASSIGNMENTS: setattr(nf, o, getattr(f,o))
            nf.__qualname__ = f"{c_.__name__}.{f.__name__}"
            
            # in class c, set function 'nf' (because copy_func) with f.__name__ as the class property
            setattr(c_, f.__name__, property(nf) if as_prop else nf)
        return f
    return _inner

## Patch function

In [ ]:
class ExistingClass(int): pass

@patch_to(ExistingClass)
def func1(x, a): return x+a

# add func1 to class

In [ ]:
t = ExistingClass(1)
assert t.func1(2)==3
assert t.func1(1)==2

Equivalent to:

In [ ]:
class ExistingClassTemp(int):
    def func1(self, a): return self+a
    
t = ExistingClassTemp(1)
assert t.func1(2)==3
assert t.func1(1)==2

You can add function f to multiple classes

In [ ]:
class ExistingClass2(int): pass
@patch_to((ExistingClass,ExistingClass2))
def func2(x, a): return x+2*a

t = ExistingClass(1)
test_eq(t.func2(1), 3)
t = ExistingClass2(1)
test_eq(t.func2(1), 3)

But we want to get the class automatically from function param instead of doing @patch_to(class_name) everytime

In [ ]:
def patch(f):
    "Decorator: add `f` to the first parameter's class (based on f's type annotations)"
    cls = next(iter(f.__annotations__.values())) # find type (class) of f's parameter
    return patch_to(cls)(f) # add function f to class cls found above

@patch
def func(x:ExistingClass, a):
    "test"
    return x+2*a

In [ ]:
print(func.__annotations__.values())
next(iter(func.__annotations__.values()))

dict_values([<class '__main__.ExistingClass'>])


__main__.ExistingClass

In [ ]:
t = ExistingClass(1)
t.func.__qualname__

'ExistingClass.func'

In [ ]:
t = ExistingClass(1)
test_eq(t.func1(2), 3)

Multiple classes

In [ ]:

@patch
def func3(x:(ExistingClass,ExistingClass2), a):
    "test"
    return x+2*a

t = ExistingClass(1)
test_eq(t.func3(2), 5)
test_eq(t.func3.__qualname__, 'ExistingClass.func3')
t = ExistingClass2(1)
test_eq(t.func3(2), 5)
test_eq(t.func3.__qualname__, 'ExistingClass2.func3')

## Patch property

In [ ]:
#export
def patch_property(f):
    "Decorator: add `f` as a property to the first parameter's class (based on f's type annotations)"
    cls = next(iter(f.__annotations__.values()))
    return patch_to(cls, as_prop=True)(f)

@patch_property
def prop(x:ExistingClass): return x+1

t = ExistingClass(1)
test_eq(t.prop, 2) # ExistingClass obj now have 'prop' as property (getter).

In [ ]:
# Equivalent to 
class ExistingClassTemp(int):
    @property
    def prop(self): return self+1
    
t = ExistingClassTemp(1)
test_eq(t.prop, 2)

# Normalization, type notation to work with tuple

In [ ]:
def norm(x,m,s): return (x-m)/s

In [ ]:
t[12:14]

tensor([[  0,   0,   0,   0,   0,   0,   0,   0, 164, 239,  26,   0,   0,   0,
           0,  96, 236,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,  12,  47,   0,   0,   0,   0,
           0, 179, 235,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]],
       dtype=torch.uint8)

In [ ]:
t = t_float # t is now float

In [ ]:
t.type()

'torch.FloatTensor'

In [ ]:
n_img = norm(t,127,50)
n_img[12:14]

tensor([[-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          0.7400,  2.2400, -2.0200, -2.5400, -2.5400, -2.5400, -2.5400, -0.6200,
          2.1800, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400],
        [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.3000, -1.6000, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,  1.0400,
          2.1600, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400]])

In [ ]:
n_img.mean()

tensor(-2.2250)

In [ ]:
@Transform
def norm(x,m,s): return (x-m)/s

In [ ]:
tupl = (t,1) # assume a image t with label '1'

# normalization is (or should be) done on the gpu, and it will be in a 'batch' form with a bunch of tuple of (image, label)

In [ ]:
temp = norm(tupl,m=127,s=50) # need keywords for @Transform norm
temp[0][12:15],temp[1] # but we don't want label to get transformed

(tensor([[-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
           0.7400,  2.2400, -2.0200, -2.5400, -2.5400, -2.5400, -2.5400, -0.6200,
           2.1800, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400, -2.5400],
         [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.3000, -1.6000, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,  1.0400,
           2.1600, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400, -2.5400],
         [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.3800,  1.9200,
           1.9000, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400, -2.5400]]), -2.52)

In [ ]:
@Transform
def norm(x:torch.Tensor,m,s): return (x-m)/s #type notation added: determine which of the tuple to apply tfm

# # equivalent to
# def _norm(x: torch.Tensor,m,s): return (x-m)/s
# norm = Transform(_norm)

In [ ]:
temp = norm(tupl,m=127,s=50)
temp[0][12:15],temp[1] # fixed!

(tensor([[-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
           0.7400,  2.2400, -2.0200, -2.5400, -2.5400, -2.5400, -2.5400, -0.6200,
           2.1800, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400, -2.5400],
         [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.3000, -1.6000, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,  1.0400,
           2.1600, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400, -2.5400],
         [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.3800,  1.9200,
           1.9000, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400, -2.5400]]), 1)

In [ ]:
tupl2 = (t,t,1)
temp2 = norm(tupl2,m=127,s=50)

In [ ]:
temp2[0][12:15],temp2[1][12:15],temp2[2]

(tensor([[-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
           0.7400,  2.2400, -2.0200, -2.5400, -2.5400, -2.5400, -2.5400, -0.6200,
           2.1800, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400, -2.5400],
         [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.3000, -1.6000, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,  1.0400,
           2.1600, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400, -2.5400],
         [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.3800,  1.9200,
           1.9000, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400, -2.5400]]),
 tensor([[-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
           0.7400,  2.2400, -2.0200, 

In [ ]:
tupl2 = ((t),(1))
temp2 = norm(tupl2,m=127,s=50)
temp2[0][12:15]

tensor([[-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          0.7400,  2.2400, -2.0200, -2.5400, -2.5400, -2.5400, -2.5400, -0.6200,
          2.1800, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400],
        [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.3000, -1.6000, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,  1.0400,
          2.1600, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400],
        [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.3800,  1.9200,
          1.9000, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400]])

In [ ]:
tupl2 = ((t,t)) # with tuple size 1, auto simplify to (t,t)
temp2 = norm(tupl2,m=127,s=50)
temp2[0][12:15]

tensor([[-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          0.7400,  2.2400, -2.0200, -2.5400, -2.5400, -2.5400, -2.5400, -0.6200,
          2.1800, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400],
        [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.3000, -1.6000, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,  1.0400,
          2.1600, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400],
        [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.3800,  1.9200,
          1.9000, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400]])

In [ ]:
# note that tuple of tuple won't work. Conclusion: @Transform meaning it will goes to each value of a tuple 
# (after tuple simplification). If value of a tuple is not torch.Tensor as described => don't transform.
tupl2 = ((t,1),t) # tuple2[0] is not torch.Tensor, tupl2[1] is
temp2 = norm(tupl2,m=127,s=50)
temp2[0][0][12:15],temp2[1][12:15]

(tensor([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 164., 239.,  26.,   0.,
            0.,   0.,   0.,  96., 236.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  12.,  47.,   0.,   0.,
            0.,   0.,   0., 179., 235.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   8., 223., 222.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.]]),
 tensor([[-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
           0.7400,  2.2400, -2.0200, -2.5400, -2.5400, -2.5400, -2.5400, -0.6200,
           2.1800, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400, -2.5400],
         [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.3000, -1.6000

In [ ]:
# equivalent to this long ass custom Transform class (good for pipeline I guess)
class NormTfm(Transform):
    def __init__(self,m,s):
        self.m,self.s = m,s
    def encodes(self,x:torch.Tensor): 
        return (x-self.m)/self.s
    
norm = NormTfm(m=127,s=50)
temp_pipe= Pipeline([norm],as_item=False)
temp =temp_pipe(tupl)

temp[0][12:15],temp[1]

(tensor([[-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
           0.7400,  2.2400, -2.0200, -2.5400, -2.5400, -2.5400, -2.5400, -0.6200,
           2.1800, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400, -2.5400],
         [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.3000, -1.6000, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,  1.0400,
           2.1600, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400, -2.5400],
         [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.3800,  1.9200,
           1.9000, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400, -2.5400]]), 1)

# Semantic markers to existing type

In [ ]:
??TensorBase

In [ ]:
class MyTensorImg(TensorBase): pass # Note: this is just a fastai wrapper class for pytorch FloatTensor
# Answer: fastai wants to add 'semantic markers' to original type to represent 'different kinds of information' (such as to show using show()),
# aside from what the original type has to offer


In [ ]:
my_t = MyTensorImg(t)

In [ ]:
type(my_t),type(t),t.type()

(__main__.MyTensorImg, torch.Tensor, 'torch.FloatTensor')

In [ ]:
@Transform
def norm(x:torch.Tensor,m,s): return (x-m)/s

In [ ]:
norm(t,m=127,s=50)[12:15]

tensor([[-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          0.7400,  2.2400, -2.0200, -2.5400, -2.5400, -2.5400, -2.5400, -0.6200,
          2.1800, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400],
        [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.3000, -1.6000, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,  1.0400,
          2.1600, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400],
        [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.3800,  1.9200,
          1.9000, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400]])

In [ ]:
norm(my_t,m=127,s=50)[12:15] # still works
#because MyTensorImage is a child of torch.Tensor, as type notation will allow children of the type specified in the func

tensor([[-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          0.7400,  2.2400, -2.0200, -2.5400, -2.5400, -2.5400, -2.5400, -0.6200,
          2.1800, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400],
        [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.3000, -1.6000, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,  1.0400,
          2.1600, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400],
        [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.3800,  1.9200,
          1.9000, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400]])

In [ ]:
@Transform
def norm2(x:MyTensorImg,m,s): return (x-m)/s 

In [ ]:
norm2(t,m=127,s=50)[12:15] # not working, since torch.Tensor is not the same or children of MyTensorImg

tensor([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 164., 239.,  26.,   0.,
           0.,   0.,   0.,  96., 236.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  12.,  47.,   0.,   0.,
           0.,   0.,   0., 179., 235.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   8., 223., 222.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.]])

In [ ]:
norm2(my_t,m=127,s=50)[12:15]

tensor([[-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          0.7400,  2.2400, -2.0200, -2.5400, -2.5400, -2.5400, -2.5400, -0.6200,
          2.1800, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400],
        [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.3000, -1.6000, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,  1.0400,
          2.1600, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400],
        [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.3800,  1.9200,
          1.9000, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400]])

# Inherit from Transform (to keep a state/attribute for your transformation + use encodes/decodes)

Remember to 'encode'

In [ ]:
class Norm(Transform):
    def __init__(self,m,s):
        super().__init__()
#         store_attr(self,'m,s')
        self.m,self.s = m,s
    def encodes(self,x:torch.Tensor): return (x-self.m)/self.s
    def encodes(self,x: MyTensorImg): return (x-self.m)/self.s
    # add other type for encodes here if you want
    
    def decodes(self,x: MyTensorImg): return (x*self.s)+ self.m

In [ ]:
# equivalent. You can add encode/decode dynamically
# class Norm(Transform):
#     def __init__(self,m,s):
#         super().__init__()
#         self.m,self.s = m,s

In [ ]:
# @Norm
# def encodes(self,x:torch.Tensor): return (x-self.m)/self.s

In [ ]:
# @Norm
# def encodes(self,x:MyTensorImg): return (x-self.m)/self.s

In [ ]:
type(my_t),type(t)

(__main__.MyTensorImg, torch.Tensor)

In [ ]:
f = Norm(m=127,s=50)
f(my_t)[12:15]

tensor([[-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          0.7400,  2.2400, -2.0200, -2.5400, -2.5400, -2.5400, -2.5400, -0.6200,
          2.1800, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400],
        [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.3000, -1.6000, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,  1.0400,
          2.1600, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400],
        [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.3800,  1.9200,
          1.9000, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400]])

In [ ]:
f(t)[12:15]

tensor([[-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          0.7400,  2.2400, -2.0200, -2.5400, -2.5400, -2.5400, -2.5400, -0.6200,
          2.1800, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400],
        [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.3000, -1.6000, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,  1.0400,
          2.1600, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400],
        [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.3800,  1.9200,
          1.9000, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400]])

In [ ]:
my_t_norm = f(my_t)

In [ ]:
f.decodes(my_t_norm)[12:15]

tensor([[  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000, 164.0000, 239.0000,  26.0000,   0.0000,   0.0000,   0.0000,
           0.0000,  96.0000, 236.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,  12.0000,  47.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000, 179.0000, 235.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           8.0000, 223.0000, 222.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000]])

# Multiple transform in one go = Pipeline

## option 1: use compose

In [ ]:
x = torch.arange(9).view(3,3)
x

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])

In [ ]:
torch.flip(x,[1])

tensor([[2, 1, 0],
        [5, 4, 3],
        [8, 7, 6]])

In [ ]:
@Transform
def flip_img(x:MyTensorImg): return torch.flip(x,[1]) # flip among axis 1

In [ ]:
f_norm =Norm(m=127,s=50)
f_compose = compose(f_norm,flip_img)

In [ ]:
temp = f_compose(my_t)
type(temp),type(my_t), temp.type(),my_t.type() # type reserve

(__main__.MyTensorImg,
 __main__.MyTensorImg,
 'torch.FloatTensor',
 'torch.FloatTensor')

Note that torch.flip won't reserve fastai type using type(), but at the @Transform decorator and it will

In [ ]:
type(torch.flip(my_t,[1])),torch.flip(my_t,[1]).type() #not reserve because torch.flip return tensor type

(torch.Tensor, 'torch.FloatTensor')

In [ ]:
type(flip_img(my_t)),flip_img(my_t).type() #force reserve

(__main__.MyTensorImg, 'torch.FloatTensor')

In [ ]:
temp[12:15]

tensor([[-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400,  2.1800, -0.6200, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.0200,  2.2400,  0.7400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400],
        [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400,  2.1600,  1.0400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -1.6000, -2.3000, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400],
        [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400,  1.9000,  1.9200, -2.3800, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400]])

In [ ]:
# but no decode
f_compose.decode

AttributeError: 'function' object has no attribute 'decode'

In [ ]:
flip_img.decode # thanks to @Transform, flip_img can still have a dummy decode function

<bound method Transform.decode of Transform: False (MyTensorImg,object) -> flip_img >

## option 2: use pipeline

In [ ]:
# note that Pipeline will convert everything into Transform, so no need for Transform annotation like in option 1

# @Transform: this is no need
def flip_img(x:MyTensorImg): return torch.flip(x,[1]) # flip among axis 1

In [ ]:
f_norm =Norm(m=127,s=50)

In [ ]:
p = Pipeline([f_norm,flip_img])
temp = p(my_t)
temp[12:15]

tensor([[-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400,  2.1800, -0.6200, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.0200,  2.2400,  0.7400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400],
        [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400,  2.1600,  1.0400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -1.6000, -2.3000, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400],
        [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400,  1.9000,  1.9200, -2.3800, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
         -2.5400, -2.5400, -2.5400, -2.5400]])

In [ ]:
type(temp),temp.type() # Pipeline also do type reserve

(__main__.MyTensorImg, 'torch.FloatTensor')

In [ ]:
type(p.decode(temp)) # reserve original type when decode as well

__main__.MyTensorImg

In [ ]:
p.decode(temp)[12:15] # note that only f_norm decode is called. flip_img does not have decode

tensor([[  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000, 236.0000,  96.0000,   0.0000,
           0.0000,   0.0000,   0.0000,  26.0000, 239.0000, 164.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000, 235.0000, 179.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,  47.0000,  12.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000, 222.0000, 223.0000,   8.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000]])

### With tuple as input

In [ ]:
type(my_t)

__main__.MyTensorImg

In [ ]:
my_t_tuple = (my_t,1)

In [ ]:
p = Pipeline([f_norm,flip_img])
temp = p(my_t_tuple)
temp[0][12:15],temp[1]

(tensor([[-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400,  2.1800, -0.6200, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.0200,  2.2400,  0.7400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400, -2.5400],
         [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400,  2.1600,  1.0400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -1.6000, -2.3000, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400, -2.5400],
         [-2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400,  1.9000,  1.9200, -2.3800, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400, -2.5400,
          -2.5400, -2.5400, -2.5400, -2.5400]]), 1)

In [ ]:
p = Pipeline([f_norm,flip_img],as_item=True) # note again: as_item=True will force the input as a whole object. So this object is gonna be type 'Tuple'
temp = p(my_t_tuple)
temp[0][12:15],temp[1] #there is no encode(...,o:Tuple), so no transformation is performed

(tensor([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 164., 239.,  26.,   0.,
            0.,   0.,   0.,  96., 236.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  12.,  47.,   0.,   0.,
            0.,   0.,   0., 179., 235.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   8., 223., 222.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.]]), 1)

# Specify output of function (not working)

In [ ]:
@Transform
def create_image_tensor(x): return tensor(array(load_image(x)))

In [ ]:
fn

PosixPath('/home/quantran/.fastai/data/mnist_tiny/train/7/9286.png')

In [ ]:
temp = create_image_tensor(fn)
type(temp),temp.type()

(torch.Tensor, 'torch.ByteTensor')

In [ ]:
# specify/cast output type (NOT WORKING NOW)
@Transform
def create_image_tensor(x)->TensorImage: return tensor(array(load_image(x)))

In [ ]:
temp = create_image_tensor(fn)
type(temp) #TODO: that does not work... it should return fastai torch_core.TensorImage

pathlib.PosixPath

In [ ]:
type(Pipeline(create_image_tensor)(fn))

torch.Tensor

In [ ]:
??TupleTransform

# Metaclass

From my own note: https://colab.research.google.com/drive/1FN8JmmY7nr0fHJ4bTkTOYOrkGQzhKyyB#scrollTo=VS4D0EoBKJVS


**Type**: a class to construct other classes

In [ ]:
class Foo(object): 
    a=1
type(Foo),type(Foo())

(type, __main__.Foo)

In [ ]:
Foo.__dict__

mappingproxy({'__module__': '__main__',
              'a': 1,
              '__dict__': <attribute '__dict__' of 'Foo' objects>,
              '__weakref__': <attribute '__weakref__' of 'Foo' objects>,
              '__doc__': None})

In [ ]:
# equivalent
Foo = type('Foo',(object,),{'a':1})
type(Foo),type(Foo())


(type, __main__.Foo)

In [ ]:
Foo.__dict__ # notice that in both cases, the class attribute 'a' is stored in __dict__

mappingproxy({'a': 1,
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Foo' objects>,
              '__weakref__': <attribute '__weakref__' of 'Foo' objects>,
              '__doc__': None})

Also note that Foo will inherit every function in type, because it's a 'type' type. Try to do Foo._ and tab for autocompletion

## Create your own 'type': a class to construct other classes

In [ ]:
def some_func2(): return 'some_func2'

In [ ]:
class _M(type):   
    @classmethod
    def __prepare__(cls,name,bases): # this is called first when you define the class with _M as its type
        # this function will prepare the class namespace, aka __dict__ object
        
        print('class method __prepare__ in _M')
        print(name,bases) #name is the name of class with _M as type
        return {'a':1,'some_func2': some_func2} 
        # this will be added to class (instance of type) dict. Also this will be input param in __new__ below (as dict) 
    
    # => to add class attributes to anything constructed by this _M (will be shown in __dict__)
    
    
    def __new__(cls,name,bases,dict): # this is called second
        print('__new__ in _M')
        print(name,bases,dict)
        return super().__new__(cls,name,bases,dict)
    


In [ ]:
class T(metaclass= _M): # construct class T using _M as a metaclass
    def some_func(self): return 'some_func'

class method __prepare__ in _M
T ()
__new__ in _M
T () {'a': 1, 'some_func2': <function some_func2 at 0x7f85bd116050>, '__module__': '__main__', '__qualname__': 'T', 'some_func': <function T.some_func at 0x7f85bd12d7a0>}


In [ ]:
T.__dict__

mappingproxy({'a': 1,
              'some_func2': <function __main__.some_func2()>,
              '__module__': '__main__',
              'some_func': <function __main__.T.some_func(self)>,
              '__dict__': <attribute '__dict__' of 'T' objects>,
              '__weakref__': <attribute '__weakref__' of 'T' objects>,
              '__doc__': None})

In [ ]:
T.a

1

In [ ]:
T.some_func

<function __main__.T.some_func(self)>

In [ ]:
print(T.some_func2()) # class function can only be called by class


some_func2


In [ ]:
print(T().some_func2()) # will return error

TypeError: some_func2() takes 0 positional arguments but 1 was given

In [ ]:
temp = T()
print(temp.a)
print(temp.some_func())

print(temp)
print(temp.__dict__)

1
some_func
{}


In [ ]:
a = {1,2}
a.add(3)
a

{1, 2, 3}